Team Members:

    Roll no. B22236 Satyam Kumar

In [78]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from scipy.linalg import pinv

In [79]:
df = pd.read_csv('ML101_train_dataset.csv')
df = df[df["Systolic BP"] > 0]
df = df[df["Diastolic BP"] > 0]

df = df.dropna(subset=['LifeStyle'])
df = df.dropna(subset=['Gender'])

classes = df['LifeStyle'].tolist()
print(len(classes))
y_train1 = list(df['Systolic BP'])
y_train2 = list(df['Diastolic BP'])

col = df.columns.tolist()
col.remove('Gender')
col.remove('LifeStyle')

imp = SimpleImputer(missing_values=np.nan, strategy='median') # replace missing numerical values with median
fit_df = pd.DataFrame(imp.fit_transform(df[col]), columns=col)
fit_df['LifeStyle'] = classes
fit_df['Systolic BP'] = y_train1
fit_df['Diastolic BP'] = y_train2   

print(fit_df)

58979
       Average Daily Steps  Hours of Sleep  Caloric Intake   Age      Height  \
0                  15725.0        6.518042          1714.0  56.0  178.416872   
1                   7424.0        4.856772          1366.0  46.0  182.281966   
2                  16484.0        4.380561          2373.0  25.0  167.868325   
3                  17563.0        4.195405          1547.0  78.0  159.352113   
4                   3700.0        9.558343          2299.0  38.0  150.136600   
...                    ...             ...             ...   ...         ...   
58974               4248.0        4.205830          3219.0  23.0  182.338167   
58975               2701.0        7.016010          2376.0  41.0  160.398936   
58976              10959.0        9.661165          2233.0  22.0  185.802297   
58977              12110.0        9.843323          1434.0  72.0  155.025779   
58978              18618.0        9.301179          3458.0  37.0  180.716378   

           Weight  Cholesterol le

Regression

In [80]:
def corr(A, B):
    N = len(A)
    mean_a = np.mean(A)
    mean_b = np.mean(B)
    s = 0
    for i in range(0, N):
         s += (A[i] - mean_a)*(B[i] - mean_b)
    s = s/N
    return s/(np.std(A)*np.std(B))

def Vandermonde(x, p):
    Z = []
    z = []
    for sample in x:
        for i in range(0, p+1):
            z.append(sample**i)
        Z.append(z)
        z = []
    return np.array(Z)

def get_parameters(x, y, p): # training data as arguments
    Z = Vandermonde(x, p)
    w_cap = np.linalg.inv(Z.T @ Z) @ Z.T @ y
    return w_cap

def regression(x, y, p):
    Z = Vandermonde(x, p)
    w_cap = np.linalg.inv(Z.T @ Z) @ Z.T @ y
    y_cap = w_cap.T @ Z.T
    return y_cap

def rmse(y_cap, y):
    mse = 0
    N = len(y_cap)
    for i in range(0, N):
        mse += (y_cap[i] - y[i])**2
    mse = mse/N
    rmse = mse**0.5
    return rmse


In [81]:
def multi_linear(X, y):
    XTX = X.T @ X
    XTy = X.T @ y
    coefficients = pinv(XTX) @ XTy

    return coefficients

def predict(coefficients, X):
    return X @ coefficients


In [82]:
X_train = fit_df.drop(['LifeStyle', 'Systolic BP', 'Diastolic BP'], axis=1)

attributes = list(X_train.columns)
pearson = []
for attribute in attributes:
        data = np.array(X_train[attribute])
        pearson.append(corr(data, y_train1))

i = np.argmax(pearson)
attribute = attributes[i] # most correlated with Systolic BP

train_data = X_train[attribute]
RMSE = []
degree = list(range(1,5))
for p in degree:
    y_cap = regression(train_data, y_train1, p)
    r = rmse(y_cap, y_train1)
    RMSE.append(r)

p1 = degree[np.argmin(RMSE)]
y_cap = regression(train_data, y_train1, p1)
w1 = get_parameters(train_data, y_train1, p1)

print("RMSE for Systolic BP:", rmse(y_cap, y_train1))

RMSE for Systolic BP: 66.70409150963651


In [83]:
attributes = list(X_train.columns)
pearson = []
for attribute in attributes:
        data = np.array(X_train[attribute])
        pearson.append(corr(data, y_train1))

i = np.argmax(pearson)
attribute = attributes[i] # most correlated with Diastolic BP

train_data = X_train[attribute]
RMSE = []
degree = list(range(1,5))
for p in degree:
    y_cap = regression(train_data, y_train2, p)
    r = rmse(y_cap, y_train2)
    RMSE.append(r)

p2 = degree[np.argmin(RMSE)]
y_cap = regression(train_data, y_train2, p2)
w2 = get_parameters(train_data, y_train2, p2)

print("RMSE for Diastolic BP:", rmse(y_cap, y_train2))

RMSE for Diastolic BP: 43.80272598595539


Predicting on the Test Data

In [84]:
test_df = pd.read_csv('ML101_dataset_test_feature.csv')
test_df = test_df.drop(columns=['Gender'])

c1 = multi_linear(X_train, y_train1)
c2 = multi_linear(X_train, y_train2)
systolic_pred = predict(c1, test_df)
diastolic_pred = predict(c2, test_df)

pred_df = pd.DataFrame()
pred_df['Systolic BP'] = systolic_pred
pred_df['Diastolic BP'] = diastolic_pred
pred_df = pred_df.rename_axis("ID")     
pred_df.to_csv('BP.csv')

Classification (Bayes Classifier Implemented)

In [85]:
import math 

def log_likelihood(X, X_mean, covmat):
    X = np.array(X)
    X_mean = np.array(X_mean)
    X_sub = X.T - X_mean.T
    covmat = np.array(covmat, dtype=float) # check covmat
    c = 1/((2*math.pi)**(4.5)) # beacuse d = 9
    d = 1/(((np.linalg.det(covmat)))**0.5)
    e1 = -0.5*((X_sub.T) @ (np.array(np.linalg.inv(covmat), dtype=float)))
    e2 = e1 @ X_sub
    exp = math.exp(e2)
    ans = np.log(c) + np.log(d) + e2*np.log(math.exp(1))
    return ans

def get_parameters(train, y):
    mean_vectors = np.zeros((4,8)) 
    covm = []
    prior = [] 

    for i in range(0, 4):  
        for j in range(0, 8):   
            values = train[train['LifeStyle'] == y[i]][attributes[j]].values  
            mean_vectors[i][j] = np.mean(values) 

    for i in range(0, 4):
        data = train[train['LifeStyle'] == y[i]].to_numpy()[:, 0:-3]
        C = np.cov(data.astype(np.float64), rowvar=False)
        covm.append(C)
        prior.append(len(data)/len(train))

    return mean_vectors, covm, prior

def bayes_multivar(train_df, test_df): 
    mean_vectors, covm, prior = get_parameters(train_df, y)

    predicted = []
    test_matrix = test_df.to_numpy()
    for sample in test_matrix:
        log_posterior = []
        for i in range(0, 4):
            log_posterior.append(np.log(prior[i])+(log_likelihood(sample, mean_vectors[i], covm[i])))
        index = np.argmax(log_posterior)
        predicted.append(y[index])
    return predicted


In [86]:
y = df['LifeStyle'].unique() 
y_pred = bayes_multivar(fit_df, test_df)

pred_classes = pd.DataFrame()
pred_classes['LifeStyle'] = y_pred
pred_classes.to_csv('LifeStyle.csv')